## Preprocessing

In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [15]:
#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("../Resources/charity_data.csv").set_index('EIN')
df.head(2)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [16]:
df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [17]:
df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [18]:
# Filter certain columns and keep only rows with the given value/s
# A status of 0 might mean ongoing loan, which might have bad data for training the model
filter_columns = [('STATUS', [1])]

# Create list of columns to drop.
# Reverse the decision from deel_learning2.  If CLASSIFICATION mostly explains each of the other parameters, the other parameters combined probably explain CLASSIFICATION
drop_columns = ['NAME', 'CLASSIFICATION', 'STATUS']


In [19]:
# First filter rows
for col, vals in filter_columns:
    df = df[df[col].isin(vals)]

# Then drop columns
# Drop columns to be dropped
df.drop(columns=drop_columns, inplace=True)


In [20]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
vc = df['APPLICATION_TYPE'].value_counts()
vc

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
app_repl = vc.loc[lambda x: x<= vc[5]].reset_index()['index']
app_repl

0      T8
1      T7
2     T10
3      T9
4     T13
5     T12
6      T2
7     T14
8     T25
9     T15
10    T29
11    T17
Name: index, dtype: object

In [23]:
# Replace in dataframe
for val in app_repl:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(val,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27032
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [25]:
df.head(2)

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,
10520599,Other,Independent,ProductDev,Association,0,N,5000,1
10531628,T3,Independent,Preservation,Co-operative,1-9999,N,108590,1


In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
dums = pd.get_dummies(df)
print(dums.shape)
dums.head(2)

(34294, 34)


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10531628,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [27]:
# Split our preprocessed data into our features and target arrays
y = dums['IS_SUCCESSFUL']
X = dums.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)

In [28]:
scale_cols = ['ASK_AMT']

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer([
        ('ct', StandardScaler(), scale_cols)
     ], remainder='passthrough')

X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.fit_transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layers = [
    {'units' : 100, 'activation': 'relu', 'input_dim': X_train_scaled.shape[1]},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 50, 'activation': 'relu'},
    {'units' : 1, 'activation': 'sigmoid'}
]

In [30]:

nn = tf.keras.models.Sequential()

for layer in layers:
    nn.add(tf.keras.layers.Dense(**layer))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3400      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 13,601
Trainable params: 13,601
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [32]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
25720/25720 [==============================] - 1s 36us/sample - loss: 0.5848 - acc: 0.7169
Epoch 2/200
25720/25720 [==============================] - 1s 30us/sample - loss: 0.5750 - acc: 0.7227
Epoch 3/200
25720/25720 [==============================] - 1s 31us/sample - loss: 0.5733 - acc: 0.7247
Epoch 4/200
25720/25720 [==============================] - 1s 30us/sample - loss: 0.5720 - acc: 0.7256
Epoch 5/200
25720/25720 [==============================] - 1s 30us/sample - loss: 0.5715 - acc: 0.7257
Epoch 6/200
25720/25720 [==============================] - 1s 32us/sample - loss: 0.5705 - acc: 0.7255
Epoch 7/200
25720/25720 [==============================] - 1s 34us/sample - loss: 0.5704 - acc: 0.7262
Epoch 8/200
25720/25720 [==============================] - 1s 32us/sample - loss: 0.5697 - acc: 0.7263
Epoch 9/200
25720/25720 [==============================] - 1s 32us/sample - loss: 0.5696 - acc: 0.7258
Epoch 10/200
25720/25720 [==============================] - 1s 31us/sampl

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8574/8574 - 0s - loss: 0.6367 - acc: 0.7257
Loss: 0.6367322610118101, Accuracy: 0.7256823182106018


In [34]:
# Export our model to HDF5 file
nn.save('no_class_filter_status_model.h5')